In [ ]:
! unzip movie_sentences.zip

Archive:  movie_sentences.zip
  inflating: imdb_indonesian_movies_2.csv  


In [ ]:
import pandas as pd

df = pd.read_csv('imdb_indonesian_movies_2.csv')
df.head()

,judul_film,ringkasan_sinopsis,genre
0,Sunan Kalijaga,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Gie,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL Movie,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Sang pencerah,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [ ]:
df = df.drop(columns='judul_film', axis=1)
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [ ]:
category = pd.get_dummies(df.genre)
df_new = pd.concat([df, category], axis=1)
df_new = df_new.drop(columns='genre', axis=1)
df_new

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [ ]:
sinopsis = df_new['ringkasan_sinopsis'].values
label = df_new[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [ ]:
print(sinopsis.shape)
print(label.shape)

(1005,)
(1005, 5)


In [ ]:
from sklearn.model_selection import train_test_split

s_train, s_test, y_train, y_test = train_test_split(sinopsis, label, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(s_train) 
tokenizer.fit_on_texts(s_test)
 
sekuens_latih = tokenizer.texts_to_sequences(s_train)
sekuens_test = tokenizer.texts_to_sequences(s_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
num_epochs = 30
history = model.fit(padded_latih, y_train, epochs=num_epochs, 
                    validation_data=(padded_test, y_test), verbose=2)

Epoch 1/30
26/26 - 29s - loss: 1.6108 - accuracy: 0.2002 - val_loss: 1.6105 - val_accuracy: 0.1841 - 29s/epoch - 1s/step
Epoch 2/30
26/26 - 25s - loss: 1.6064 - accuracy: 0.2637 - val_loss: 1.6114 - val_accuracy: 0.2189 - 25s/epoch - 975ms/step
Epoch 3/30
26/26 - 26s - loss: 1.5709 - accuracy: 0.3259 - val_loss: 1.6226 - val_accuracy: 0.2239 - 26s/epoch - 1s/step
Epoch 4/30
26/26 - 25s - loss: 1.4060 - accuracy: 0.4104 - val_loss: 1.6550 - val_accuracy: 0.2537 - 25s/epoch - 943ms/step
Epoch 5/30
26/26 - 26s - loss: 1.1242 - accuracy: 0.5162 - val_loss: 2.4509 - val_accuracy: 0.2239 - 26s/epoch - 985ms/step
Epoch 6/30
26/26 - 24s - loss: 0.8144 - accuracy: 0.6244 - val_loss: 2.2255 - val_accuracy: 0.2040 - 24s/epoch - 940ms/step
Epoch 7/30
26/26 - 25s - loss: 0.5694 - accuracy: 0.7450 - val_loss: 3.0454 - val_accuracy: 0.2289 - 25s/epoch - 962ms/step
Epoch 8/30
26/26 - 26s - loss: 0.4063 - accuracy: 0.8545 - val_loss: 3.0972 - val_accuracy: 0.2239 - 26s/epoch - 1s/step
Epoch 9/30
26/26 